In [ ]:
import utils
from matplotlib import pyplot as plt
import fastf1
import pandas as pd
from datetime import timedelta, datetime

In [ ]:
magColor = fastf1.plotting.driver_color("mag")
verColor = fastf1.plotting.driver_color("ver")

In [ ]:
session = fastf1.get_session(2022, 21, 'Q')
session.load(laps=True, telemetry=True, weather=True)
laps = session.laps

q1, q2, q3 = laps.split_qualifying_sessions()
wheater = q3.get_weather_data()

fast_mag = q3.pick_driver('MAG')
mag_car_data = fast_mag.get_car_data()
t_mag = mag_car_data['SessionTime']
vCar_mag = mag_car_data['Speed']

fast_ver = q3.pick_driver('VER')
ver_car_data = fast_ver.get_car_data()
t_ver = ver_car_data['SessionTime']
vCar_ver = ver_car_data['Speed']

In [ ]:
best_lap_mag, best_lap_ver = utils.get_qualifying_laps_for_mag_and_ver(q3)

In [ ]:
df_weather = pd.read_csv("data/q3_wheater.csv")
track_status = session.track_status
yellow_flag = track_status[track_status["Message"] == "Yellow"]
yellow_time = utils.convert_delta_to_time(yellow_flag["Time"])

red_flag = track_status[track_status["Message"] == "Red"].iloc[0]
red_time = utils.convert_delta_to_time(red_flag["Time"])

allClear = track_status[track_status["Message"] == "AllClear"]
allClear_time = utils.convert_delta_to_time(allClear["Time"])
allClearAfterRed = allClear.iloc[-1]
allClearAfterRed_time = utils.convert_delta_to_time(allClearAfterRed["Time"])

In [ ]:
min_time = df_weather["Time"].min()
min_time = utils.convert_string_to_time(min_time)
max_time = df_weather["Time"].max()
max_time = utils.convert_string_to_time(max_time)
td_5sec = timedelta(seconds=5)

In [ ]:
## track temp over time
fig, ax = plt.subplots()
ax.xaxis.axis_date()

red_flag = track_status[track_status["Message"] == "Red"].iloc[0]
red_time = utils.convert_delta_to_time(red_flag["Time"])

ax.vlines(x=yellow_time, ymin=25, ymax=27, color="gold", linewidth=3)
while red_time < allClearAfterRed_time:
    red_time += td_5sec
    ax.vlines(x=red_time, ymin=25, ymax=27, color="red", linewidth=2)
ax.vlines(x=allClearAfterRed_time, ymin=25, ymax=27, color="green", linewidth=4)

for idx, _  in enumerate(df_weather.index):

    isRainfall = df_weather.iloc[idx]["Rainfall"]
    timeString = df_weather.iloc[idx]["Time"]
    trackTemp = df_weather.iloc[idx]["TrackTemp"]
    timeStamp = utils.convert_string_to_time(timeString)
    ax.plot(timeStamp, trackTemp, c="blue" if isRainfall else "black",  marker="o", markersize=5)

In [ ]:
best_lap_mag_date_time = best_lap_mag["Time"].apply(utils.convert_delta_to_time)
best_lap_ver_date_time = best_lap_ver["Time"].apply(utils.convert_delta_to_time) 

In [ ]:
fig, ax = plt.subplots()
ax.plot(best_lap_mag_date_time, best_lap_mag["Speed"], c="red")
ax.plot(best_lap_ver_date_time, best_lap_ver["Speed"], c="blue")
ax.set_xlabel("lap time m:ss")
ax.set_ylabel("Speed (km/h)")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.xaxis.axis_date()

ax.set_title("Speed in qualifying")
plt.setp(ax.get_xticklabels()[::2], visible=False)